In [ ]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets
!pip install -q scann

In [1]:
import numpy as np 
import pandas as pd 
import os
import zipfile
import json
import tensorflow as tf
import tensorflow_recommenders as tfrs
import csv
from tqdm import tqdm
from typing import Dict, Text

2022-12-30 09:09:51.547612: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-30 09:09:52.574716: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-12-30 09:09:52.574933: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Co

In [2]:
print('TensorFlow version: {}'.format(tf.__version__))
print('TensorFlow Recommender version: {}'.format(tfrs.__version__))
print('TensorFlow Raking')
print('TensorFlow ScaNN')

TensorFlow version: 2.11.0
TensorFlow Recommender version: v0.7.2
TensorFlow Raking
TensorFlow ScaNN


In [3]:
TEST_FILE = '/kaggle/input/otto-recommender-system/test.jsonl'
SCANN_DIR = '/kaggle/input/otto-basemodel-v1/kaggle/working/serving_model/ScaNN_Model'
MODEL_DIR = '/kaggle/input/otto-basemodel-v1/kaggle/working/serving_model/Norm_Model'
SUBMIT_FILE = '/kaggle/working/submission_file4.csv'
OUTPUT_DIR = '/kaggle/working/'

# **Helper Functions**

In [4]:
def preprocess(df):
    initial_data, features={}, {}
    session_list, item_A_list, ts_list = [], [], []
   
    for index in df.index:
        session_id = str(df['session'][index])
        session_list.append(session_id)
        item_A_list.append(str(df['events'][index][0]['type']))
        ts_list.append(df['events'][index][0]['ts'])
        click_list, cart_list, order_list = [], [], []
    
        for event in df['events'][index]:
            if event['type']=='clicks':
                click_list.append(event['aid'])
            elif event['type']=='carts':
                cart_list.append(event['aid'])
            else:
                order_list.append(event['aid'])
        initial_data[session_id] = {'clicks':click_list, 'carts':cart_list, 'orders':order_list}
    
    features={'session_list':session_list, 'item_A_list':item_A_list, 'ts_list':ts_list}
    return features, initial_data

def get_rating(item_A, item_B_list, ts):
    ratings = {}
    for item_B in item_B_list:
        A, B, ratings[item_B] = rat_model({
        "item_A": np.array([item_A]),
        "item_B": np.array([item_B]),
        "ts":np.array([ts])
            })
    return {k: v for k, v in sorted(ratings.items(), key=lambda item: item[1],reverse=True)}
        
def postprocess(i_aid, aid_rating):
    predict_click_list=[]
    predict_cart_list=[]
    predict_order_list=[]
    final_aid={}
    
    filter_aid = dict(filter(lambda elem: elem[1] > 1.5,aid_rating.items()))
    for key, value in filter_aid.items():
        if value.numpy() > 2.999:
            predict_click_list.append(key.decode())
            predict_cart_list.append(key.decode())
            predict_order_list.append(key.decode())
        
        elif 2.2 <= value.numpy() <= 2.999:
            predict_click_list.append(key.decode())
            predict_cart_list.append(key.decode())
        
        elif value.numpy() >= 1.2:
            predict_click_list.append(key.decode())
        
        else:
            pass
    
    final_click_list = i_aid['clicks'] + predict_click_list
    final_cart_list = i_aid['carts'] + predict_cart_list
    final_order_list = i_aid['orders'] + predict_order_list
    
    final_aid['clicks'] = final_click_list
    final_aid['carts'] = final_cart_list
    final_aid['orders'] = final_order_list
        
    return final_aid

def writefile(record, header):
    with open(SUBMIT_FILE, 'a') as file:
        writer = csv.writer(file)
        if header:
            writer.writerow(record)
        else:
            writer.writerows(record)
    return

In [5]:
ret_model = tf.saved_model.load(SCANN_DIR)
rat_model = tf.keras.models.load_model(MODEL_DIR)

2022-12-30 09:10:06.652112: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 237517312 exceeds 10% of free system memory.
2022-12-30 09:10:23.233361: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 237517312 exceeds 10% of free system memory.
2022-12-30 09:10:23.233412: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 237517312 exceeds 10% of free system memory.


In [6]:
%%time
chunk_size=100000
num_lines = sum(1 for line in open(TEST_FILE))
num_chunks= int(np.ceil(num_lines / chunk_size))

print('Number of lines: ',num_lines)
print('Number of chunks: ',num_chunks)

Number of lines:  1671803
Number of chunks:  17
CPU times: user 525 ms, sys: 199 ms, total: 723 ms
Wall time: 4.69 s


In [7]:
%%time
chunks = pd.read_json(TEST_FILE, lines=True, chunksize=chunk_size)

header_record=['session_type, labels']
header_flag=True
writefile(header_record, header_flag)

CPU times: user 1.88 ms, sys: 1.16 ms, total: 3.04 ms
Wall time: 3.14 ms


In [8]:
%%time
header_flag=False
for chunk_num, chunk in enumerate(chunks):
    submit_records = []
    rating={}
    print('Processing chunk # : ', chunk_num)
    
    features, initial_data = preprocess(chunk)
    _, retrieved_data = ret_model(tf.constant(features['item_A_list']))
    
#     for i in range(len(features['item_A_list'])):
#         item_A = [features['item_A_list'][i]] * len(retrieved_data[i]) 
#         item_B = retrieved_data[i]
#         ts = [features['ts_list'][i]] * len(retrieved_data[i])
        
#         embed_A, embed_B, rating[i] =  rat_model({
#             "item_A": np.array(item_A),
#             "item_B": np.array(item_B),
#             "ts":np.array(ts)
#             })
        
    print(len(retrieved_data))
#     break
    
    
    
#     for index in chunk.index:
#         events = chunk['events'][index]
#         session_id = str(chunk['session'][index])
#         first_aid =  str(events[0]['aid'])
#         session_start_ts = events[0]['ts']
#         print('first_aid', first_aid)
                                
#         i_aid = preprocess(events)
#         _, p_aid_list = ret_model(tf.constant([first_aid]))
#         aid_rating = get_rating(first_aid, p_aid_list[0].numpy(), session_start_ts)
#         f_aid = postprocess(i_aid, aid_rating)
        
#         for key, value in f_aid.items():
#             temp_list=[]
#             if len(value) > 0:
#                 session_type = session_id + '_' + str(key)
#                 f_aid_list = " ".join(value)
#                 temp_list.append(session_type)
#                 temp_list.append(f_aid_list)
#                 submit_records.append(temp_list)    
        
#         print(submit_records)
#         break;
# #     writefile(submit_records, header_flag)
# #     break

Processing chunk # :  0
100000
Processing chunk # :  1
100000
Processing chunk # :  2
100000
Processing chunk # :  3
100000
Processing chunk # :  4
100000
Processing chunk # :  5
100000
Processing chunk # :  6
100000
Processing chunk # :  7
100000
Processing chunk # :  8
100000
Processing chunk # :  9
100000
Processing chunk # :  10
100000
Processing chunk # :  11
100000
Processing chunk # :  12
100000
Processing chunk # :  13
100000
Processing chunk # :  14
100000
Processing chunk # :  15
100000
Processing chunk # :  16
71803
CPU times: user 3min 47s, sys: 780 ms, total: 3min 48s
Wall time: 2min 37s


In [ ]:
_, p_aid_list = ret_model(tf.constant(['59625','59625','99163']))
print(p_aid_list)        
        

In [ ]:
A, B, ratings = rat_model({
        "item_A": np.array(['59625','59625','59625','59625']),
        "item_B": np.array(['1702672','725615','1702672','725615']),
        "ts":np.array([1234, 1234,1234, 1234])
            })

print(ratings)